In [3]:
import numpy as np
import pandas as pd

# read in hh using list comprehension
hh = [line.rstrip('\n') for line in open('hh.txt')]
# display first 10 list items
hh

['Winning Poker Network Game #1081326940: No Limit Holdem (25/50) [2017/12/22 17:59:45 UTC]',
 'Table: $10 Regular 9-Max, Table 1',
 'Tournament: 8140214',
 'Seats: 9',
 'Seat 1: W1renut (1,439)',
 'Seat 2: Truck1536 (3,141)',
 'Seat 3: allegro88 (1,670)',
 'Seat 5: unclepete55 (1,721)',
 'Seat 6: Cutch123 (1,310)',
 'Seat 7: cobblestoner (1,201)',
 'Seat 8: otlichno (1,215)',
 'Seat 9: O.G. SKARFINGER (1,803)',
 'Button is seat 8',
 'O.G. SKARFINGER: posts small blind 25',
 'W1renut: posts big blind 50',
 '*** HOLE CARDS ***',
 'unclepete55: dealt [Qh Qs]',
 'Truck1536: folds',
 'allegro88: folds',
 'unclepete55: calls 50',
 'Cutch123: folds',
 'cobblestoner: folds',
 'otlichno: raises 200',
 'O.G. SKARFINGER: folds',
 'W1renut: raises 300',
 'unclepete55: folds',
 'otlichno: raises all-in 1,015',
 'W1renut: calls 865',
 '*** FLOP *** [Qc Ah 8s]',
 '*** TURN *** [Qc Ah 8s] [3h]',
 '*** RIVER *** [Qc Ah 8s] [3h] [2h]',
 '*** SUMMARY ***',
 'Pot: 2,505 | Rake: 0 |',
 'Board: [Qc Ah 8s 3

In [48]:
def get_stacks():
    # line.split()[1] is a string that i slice the first char with [:1]
    seats = [line.split()[1][:1] for line in hh if '(' in line]
    # convert seats to int and drop first and last items in seats list
    seats = [int(seat) for seat in seats[1:-1]]
    
    # get names associated with each seat
    names = [line.split()[2:-1] for line in hh if '(' in line]
    names = [name for name in names[1:-1]]
    
    # get stacks associated with each seat
    stacks = [line.split()[-1] for line in hh if '(' in line]
    # drop parenthesis on each stack and drop the comma and convert stack to int
    # note: dropping first and last items in list as in seats
    stacks = [int(stack[1:-1].replace(',', '')) for stack in stacks[1:-1]]
    
    # identify which seat is the button
    # note this list comprehension only returns a one element list which I extract
    btn = [int(line.split()[-1]) for line in hh if 'Button is seat ' in line][0]
    
    index_btn = seats.index(btn)
    
    # create position list in appropriate deal order for 9 players PT4 convention
    pos = [6,5,4,3,2,1,0,9,8]
    pos = pos[9-len(stacks):]
    num_stacks = len(stacks)
    # repeat list to create a window
    stacks1 = (stacks * 2)[index_btn:index_btn+num_stacks]
    names1 = (names * 2)[index_btn:index_btn+num_stacks]
    # join stacks to match deal order and positions
    stacks2 = stacks1[3:] + stacks1[:3]
    names2 = names1[3:] + names1[:3]
    df = pd.DataFrame({'pos': pos, 'name': names2, 'chips': stacks2}).set_index('pos')
    
    return df

get_stacks()


,name,chips
pos,,
5,[Truck1536],3141
4,[allegro88],1670
3,[unclepete55],1721
2,[Cutch123],1310
1,[cobblestoner],1201
0,[otlichno],1215
9,"[O.G., SKARFINGER]",1803
8,[W1renut],1439


In [8]:
def get_bb():
    bb = [int(line.split()[-1]) for line in hh if 'posts big blind' in line][0]
    return bb

get_bb()

50

In [11]:
def get_sb():
    sb = [int(line.split()[-1]) for line in hh if 'posts small blind' in line][0]
    return sb

get_sb()

25

In [125]:
# for the purpose of my study I am calculating 9-man sng payout structure
# 0.5, 0.3, 0.2 to 1st, 2nd, 3rd respectively
def ICM9(df):
    payouts = [0.5, 0.3, 0.2]
    tot = df.chips.sum()
    length = len(df.chips)
    df['p1st'] = df.chips / tot
    # df.left1 is the amount of chips remaining if that player wins 1st place
    df['left1'] = tot - df.chips
    
    # left2 is a df of chips remaining if players i and j get 1st and 2nd
    # note that who gets first and who gets second is irrelevant since the chips remaining
    #      will be the same
    left2 = pd.DataFrame(np.nan, index = range(length), columns = range(len(df.chips)))
    for i in range(len(df.chips)-1):
        for j in range(i+1,len(df.chips)):
            left2.iloc[i,j] = tot - df.chips.iloc[i] - df.chips.iloc[j]
    
    
    p2 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
    # i is the row and represents the player placing first in the tournament
    for i in range(length):
        # j is the player's equity in 2nd place given player i is the winner
        for j in range(length):
            # if i == j then player cannot be second because she has already won
            if i != j:
                p2.iloc[i,j] = df.p1st.iloc[i] * df.chips.iloc[j] / df.left1.iloc[i] 
    
    return resa

res = ICM9(get_stacks())


res


[0    0.060359
 1    0.038105
 2    0.039080
 3    0.030872
 4    0.028565
 5    0.028865
 6    0.040620
 7    0.033533
 dtype: float64, 0    0.060359
 1    0.026853
 2    0.027484
 3    0.022045
 4    0.020473
 5    0.020678
 6    0.028471
 7    0.023837
 dtype: float64, 0    0.050506
 1    0.026853
 2    0.022085
 3    0.017936
 4    0.016705
 5    0.016866
 6    0.022815
 7    0.019323
 dtype: float64, 0    0.040307
 1    0.021431
 2    0.022085
 3    0.013682
 4    0.012806
 5    0.012921
 6    0.016961
 7    0.014651
 dtype: float64, 0    0.032806
 1    0.017442
 2    0.017975
 3    0.013682
 4    0.009938
 5    0.010020
 6    0.012656
 7    0.011214
 dtype: float64, 0    0.025990
 1    0.013818
 2    0.014240
 3    0.010840
 4    0.009938
 5    0.007383
 6    0.008743
 7    0.008092
 dtype: float64, 0    0.019087
 1    0.010148
 2    0.010458
 3    0.007960
 4    0.007298
 5    0.007383
 6    0.004780
 7    0.004929
 dtype: float64, 0    0.008328
 1    0.004428
 2    0.004563
 3 